###*Sayantan Mukherjee 60009220131*

In [ ]:
import numpy as np

class GridWorld:
    def __init__(self):
        self.grid_size = 3
        self.num_states = self.grid_size * self.grid_size
        self.num_actions = 4
        self.actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        self.gamma = 0.9
        self.grid_rewards = np.zeros((self.grid_size, self.grid_size))
        self.grid_rewards[self.grid_size - 1, self.grid_size - 1] = 1  #goal reward
        self.transition_prob = self._build_transition_prob()

    def _build_transition_prob(self):
        transition_prob = np.zeros((self.num_states, self.num_actions, self.num_states))
        for s in range(self.num_states):
            for a in range(self.num_actions):
                next_states = self._get_next_states(s, a)
                for next_state, prob in next_states.items():
                    transition_prob[s, a, next_state] = prob
        return transition_prob

    def _get_next_states(self, state, action):
        i, j = state // self.grid_size, state % self.grid_size
        next_states = {}
        if action == 0:
            next_i = max(i - 1, 0)
            next_states[next_i * self.grid_size + j] = 1
        elif action == 1:
            next_i = min(i + 1, self.grid_size - 1)
            next_states[next_i * self.grid_size + j] = 1
        elif action == 2:
            next_j = max(j - 1, 0)
            next_states[i * self.grid_size + next_j] = 1
        elif action == 3:
            next_j = min(j + 1, self.grid_size - 1)
            next_states[i * self.grid_size + next_j] = 1
        return next_states

    def iterative_policy_eval(self, policy, theta=0.0001, max_iterations=1000):
        V = np.zeros(self.num_states)
        for _ in range(max_iterations):
            delta = 0
            for s in range(self.num_states):
                v = V[s]
                if s == (self.num_states - 1):
                    V[s] = self.grid_rewards[s // self.grid_size, s % self.grid_size]
                else:
                    a = policy[s]
                    V[s] = sum([self.transition_prob[s, a, s1] *
                                (self.grid_rewards[s1 // self.grid_size, s1 % self.grid_size] +
                                 self.gamma * V[s1]) for s1 in range(self.num_states)])
                delta = max(delta, abs(v - V[s]))
            if delta < theta:
                break
        return V

    def policy_itr(self, max_iterations=100):
        policy = np.random.randint(0, self.num_actions, self.num_states)
        for _ in range(max_iterations):
            V = self.iterative_policy_eval(policy)
            policy_stable = True
            for s in range(self.num_states):
                old_action = policy[s]
                policy[s] = np.argmax([sum([self.transition_prob[s, a, s1] *
                                             (self.grid_rewards[s1 // self.grid_size, s1 % self.grid_size] +
                                              self.gamma * V[s1]) for s1 in range(self.num_states)]) for a in range(self.num_actions)])
                if old_action != policy[s]:
                    policy_stable = False
            if policy_stable:
                break
        return policy

    def value_itr(self, theta=0.0001, max_iterations=1000):
        V = np.zeros(self.num_states)
        for _ in range(max_iterations):
            delta = 0
            for s in range(self.num_states):
                v = V[s]
                if s == (self.num_states - 1):
                    V[s] = self.grid_rewards[s // self.grid_size, s % self.grid_size]
                else:
                    V[s] = max([sum([self.transition_prob[s, a, s1] *
                                     (self.grid_rewards[s1 // self.grid_size, s1 % self.grid_size] +
                                      self.gamma * V[s1]) for s1 in range(self.num_states)]) for a in range(self.num_actions)])
                delta = max(delta, abs(v - V[s]))
            if delta < theta:
                break
        policy = np.zeros(self.num_states, dtype=int)
        for s in range(self.num_states):
            policy[s] = np.argmax([sum([self.transition_prob[s, a, s1] *
                                         (self.grid_rewards[s1 // self.grid_size, s1 % self.grid_size] +
                                          self.gamma * V[s1]) for s1 in range(self.num_states)]) for a in range(self.num_actions)])
        return policy


if __name__ == "__main__":
    grid_world = GridWorld()

    policy = np.random.randint(0, grid_world.num_actions, grid_world.num_states)
    V = grid_world.iterative_policy_eval(policy)
    print("Iterative Policy Evaluation:")
    print(V.reshape(grid_world.grid_size, grid_world.grid_size))

    policy = grid_world.policy_itr()
    print("\nPolicy Iteration:")
    print(policy.reshape(grid_world.grid_size, grid_world.grid_size))

    policy = grid_world.value_itr()
    print("\nValue Iteration:")
    print(policy.reshape(grid_world.grid_size, grid_world.grid_size))


Iterative Policy Evaluation:
[[0.   0.   0.  ]
 [0.   1.71 1.9 ]
 [0.   0.   1.  ]]

Policy Iteration:
[[1 1 1]
 [1 1 1]
 [3 3 1]]

Value Iteration:
[[1 1 1]
 [1 1 1]
 [3 3 1]]



- **Setup**: A simple 3x3 grid with a goal at (2,2) offering a reward of 1, discount factor (gamma) of 0.9, and four actions (UP, DOWN, LEFT, RIGHT). No obstacles.
- **Algorithms**:
  - **Iterative Policy Evaluation**: Evaluates a random policy, resulting in a value function reflecting expected rewards (e.g., highest value near the goal at 1.9).
  - **Policy Iteration**: Iteratively improves a random policy, converging to an optimal policy (mostly moving RIGHT and DOWN toward the goal).
  - **Value Iteration**: Directly computes the optimal value function and derives the policy, yielding identical results to Policy Iteration.
- **Results**:
  - Value function shows increasing values toward the goal.
  - Policies: 0=UP, 1=DOWN, 2=LEFT, 3=RIGHT; optimal policy directs all moves toward (2,2).

In [ ]:
import numpy as np

class GridWorld:
    def __init__(self, grid_size=16):
        if (grid_size & (grid_size - 1)) != 0:
            raise ValueError("grid_size must be a power of 2 (e.g., 2, 4, 8, 16, ...)")
        self.grid_size = grid_size
        self.num_states = self.grid_size * self.grid_size
        self.num_actions = 4
        self.actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        self.gamma = 0.9
        self.grid_rewards = np.zeros((self.grid_size, self.grid_size))
        self.grid_rewards[self.grid_size - 1, self.grid_size - 1] = 1  # Goal state
        self.transition_prob = self._build_transition_prob()

    def _build_transition_prob(self):
        transition_prob = np.zeros((self.num_states, self.num_actions, self.num_states))
        for s in range(self.num_states):
            for a in range(self.num_actions):
                next_states = self._get_next_states(s, a)
                for next_state, prob in next_states.items():
                    transition_prob[s, a, next_state] = prob
        return transition_prob

    def _get_next_states(self, state, action):
        i, j = state // self.grid_size, state % self.grid_size
        next_states = {}
        if action == 0:  # UP
            next_i = max(i - 1, 0)
            next_states[next_i * self.grid_size + j] = 1
        elif action == 1:  # DOWN
            next_i = min(i + 1, self.grid_size - 1)
            next_states[next_i * self.grid_size + j] = 1
        elif action == 2:  # LEFT
            next_j = max(j - 1, 0)
            next_states[i * self.grid_size + next_j] = 1
        elif action == 3:  # RIGHT
            next_j = min(j + 1, self.grid_size - 1)
            next_states[i * self.grid_size + next_j] = 1
        return next_states

    def iterative_policy_eval(self, policy, theta=1e-4, max_iterations=1000):
        V = np.zeros(self.num_states)
        iterations = 0
        for _ in range(max_iterations):
            iterations += 1
            delta = 0
            for s in range(self.num_states):
                if s == self.num_states - 1:  # Skip terminal state
                    continue
                v = V[s]
                a = policy[s]
                V[s] = sum(
                    self.transition_prob[s, a, s1] *
                    (self.grid_rewards[s1 // self.grid_size, s1 % self.grid_size] + self.gamma * V[s1])
                    for s1 in range(self.num_states)
                )
                delta = max(delta, abs(v - V[s]))
            if delta < theta:
                break
        return V, iterations

    def policy_itr(self, theta=1e-4, max_iterations=1000):
        policy = np.random.randint(0, self.num_actions, self.num_states)
        policy_stable = False
        total_eval_iterations = 0  # Track total inner iterations
        outer_iterations = 0

        while not policy_stable and outer_iterations < max_iterations:
            outer_iterations += 1
            V, eval_iterations = self.iterative_policy_eval(policy, theta, max_iterations)
            total_eval_iterations += eval_iterations
            policy_stable = True
            for s in range(self.num_states):
                if s == self.num_states - 1:  # Skip terminal state
                    continue
                old_action = policy[s]
                action_values = np.zeros(self.num_actions)
                for a in range(self.num_actions):
                    action_values[a] = sum(
                        self.transition_prob[s, a, s1] *
                        (self.grid_rewards[s1 // self.grid_size, s1 % self.grid_size] + self.gamma * V[s1])
                        for s1 in range(self.num_states)
                    )
                policy[s] = np.argmax(action_values)
                if old_action != policy[s]:
                    policy_stable = False
        return policy, V, total_eval_iterations

    def value_itr(self, theta=1e-4, max_iterations=1000):
        V = np.zeros(self.num_states)
        iterations = 0
        while iterations < max_iterations:
            iterations += 1
            delta = 0
            for s in range(self.num_states):
                if s == self.num_states - 1:  # Skip terminal state
                    continue
                v = V[s]
                V[s] = max(
                    sum(
                        self.transition_prob[s, a, s1] *
                        (self.grid_rewards[s1 // self.grid_size, s1 % self.grid_size] + self.gamma * V[s1])
                        for s1 in range(self.num_states)
                    )
                    for a in range(self.num_actions)
                )
                delta = max(delta, abs(v - V[s]))
            if delta < theta:
                break
        policy = np.zeros(self.num_states, dtype=int)
        for s in range(self.num_states):
            action_values = np.zeros(self.num_actions)
            for a in range(self.num_actions):
                action_values[a] = sum(
                    self.transition_prob[s, a, s1] *
                    (self.grid_rewards[s1 // self.grid_size, s1 % self.grid_size] + self.gamma * V[s1])
                    for s1 in range(self.num_states)
                )
            policy[s] = np.argmax(action_values)
        return policy, V, iterations

if __name__ == "__main__":
    grid_size = 16
    grid_world = GridWorld(grid_size=grid_size)

    # ---- Iterative Policy Evaluation ----
    random_policy = np.random.randint(0, grid_world.num_actions, grid_world.num_states)
    V_iter, eval_iterations = grid_world.iterative_policy_eval(random_policy)
    print("Iterative Policy Evaluation:")
    print(V_iter.reshape(grid_world.grid_size, grid_world.grid_size))
    print(f"Converged in {eval_iterations} iterations\n")

    # ---- Policy Iteration ----
    policy_pi, V_pi, pi_total_iter = grid_world.policy_itr()
    print("Policy Iteration:")
    print("Policy (0=UP, 1=DOWN, 2=LEFT, 3=RIGHT):")
    print(policy_pi.reshape(grid_world.grid_size, grid_world.grid_size))
    print("Value Function:")
    print(V_pi.reshape(grid_world.grid_size, grid_world.grid_size))
    print(f"Total iterations (including inner evaluations): {pi_total_iter}\n")

    # ---- Value Iteration ----
    policy_vi, V_vi, vi_iterations = grid_world.value_itr()
    print("Value Iteration:")
    print("Policy (0=UP, 1=DOWN, 2=LEFT, 3=RIGHT):")
    print(policy_vi.reshape(grid_world.grid_size, grid_world.grid_size))
    print("Value Function:")
    print(V_vi.reshape(grid_world.grid_size, grid_world.grid_size))
    print(f"Total iterations: {vi_iterations}\n")

    # ---- Fair Comparison ----
    if pi_total_iter < vi_iterations:
        print("Policy Iteration converged faster in total iterations.")
    elif pi_total_iter > vi_iterations:
        print("Value Iteration converged faster in total iterations.")
    else:
        print("Both methods converged in the same number of iterations.")

Iterative Policy Evaluation:
[[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. 

- **Setup**: Expanded to a 16x16 grid, still with a goal at (15,15) and no obstacles. Grid size must be a power of 2.
- **Algorithms**: Same as above, with added iteration tracking for comparison.
- **Results**:
  - **Iterative Policy Evaluation**: Converges in 3 iterations with a random policy, showing low values due to poor initial policy.
  - **Policy Iteration**: Converges with 496 total iterations (including inner evaluations), optimal policy moves DOWN then RIGHT to the goal, value function increases smoothly toward 1.0.
  - **Value Iteration**: Converges in 31 iterations, producing the same optimal policy and value function.
- **Comparison**: Value Iteration is faster (31 vs. 496 iterations), highlighting its efficiency in larger grids without obstacles.

In [ ]:
import numpy as np

class GridWorld:
    def __init__(self, grid_size=4):
        if (grid_size & (grid_size - 1)) != 0:
            raise ValueError("grid_size must be a power of 2 (e.g., 2, 4, 8, 16, ...)")
        self.grid_size = grid_size
        self.num_states = self.grid_size * self.grid_size
        self.num_actions = 4
        self.actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        self.gamma = 0.9

        # Initialize grid rewards and obstacles
        self.grid_rewards = np.zeros((self.grid_size, self.grid_size))
        self.grid_obstacles = np.zeros((self.grid_size, self.grid_size), dtype=bool)
        self._generate_obstacles()

        # Goal state (bottom-right)
        self.goal = (self.grid_size - 1, self.grid_size - 1)
        self.grid_rewards[self.goal] = 1

        # Build transition matrices
        self.transition_prob, self.transition_reward = self._build_transition_matrices()

    def _generate_obstacles(self):
        """Generate random obstacles covering 15% of the grid (excluding start and goal)"""
        num_obstacles = int(0.15 * self.grid_size**2)
        start = (0, 0)
        goal = (self.grid_size-1, self.grid_size-1)
        available = []

        for i in range(self.grid_size):
            for j in range(self.grid_size):
                if (i, j) != start and (i, j) != goal:
                    available.append((i, j))

        for idx in np.random.choice(len(available), size=num_obstacles, replace=False):
            i, j = available[idx]
            self.grid_obstacles[i, j] = True

    def _get_next_state(self, state, action):
        """Returns (next_state, reward) for a given state-action pair"""
        i, j = state // self.grid_size, state % self.grid_size

        # Calculate intended movement
        if action == 0:  # UP
            ni, nj = max(i-1, 0), j
        elif action == 1:  # DOWN
            ni, nj = min(i+1, self.grid_size-1), j
        elif action == 2:  # LEFT
            ni, nj = i, max(j-1, 0)
        else:  # RIGHT
            ni, nj = i, min(j+1, self.grid_size-1)

        # Check for obstacle collision
        if self.grid_obstacles[ni, nj]:
            return state, -0.1  # Penalize hitting obstacle
        else:
            next_state = ni * self.grid_size + nj
            return next_state, self.grid_rewards[ni, nj]

    def _build_transition_matrices(self):
        """Build transition probability and reward matrices"""
        transition_prob = np.zeros((self.num_states, self.num_actions, self.num_states))
        transition_reward = np.zeros((self.num_states, self.num_actions, self.num_states))

        for s in range(self.num_states):
            for a in range(self.num_actions):
                next_state, reward = self._get_next_state(s, a)
                transition_prob[s, a, next_state] = 1.0
                transition_reward[s, a, next_state] = reward

        return transition_prob, transition_reward

    def iterative_policy_eval(self, policy, theta=1e-4, max_iter=1000):
        V = np.zeros(self.num_states)
        iterations = 0
        for _ in range(max_iter):
            iterations += 1
            delta = 0
            for s in range(self.num_states):
                if s == self.num_states - 1:  # Skip terminal state
                    continue
                v_old = V[s]
                a = policy[s]
                V[s] = sum(
                    self.transition_prob[s, a, s1] *
                    (self.transition_reward[s, a, s1] + self.gamma * V[s1])
                    for s1 in range(self.num_states)
                )
                delta = max(delta, abs(v_old - V[s]))
            if delta < theta:
                break
        return V, iterations

    def policy_itr(self, theta=1e-4, max_iter=1000):
        policy = np.random.randint(0, self.num_actions, self.num_states)
        policy_stable = False
        total_iterations = 0
        outer_iterations = 0

        while not policy_stable and outer_iterations < max_iter:
            outer_iterations += 1
            # Policy Evaluation
            V, eval_iterations = self.iterative_policy_eval(policy, theta, max_iter)
            total_iterations += eval_iterations

            # Policy Improvement
            policy_stable = True
            for s in range(self.num_states):
                if s == self.num_states - 1:  # Skip terminal state
                    continue
                old_action = policy[s]
                q_values = np.zeros(self.num_actions)
                for a in range(self.num_actions):
                    q_values[a] = sum(
                        self.transition_prob[s, a, s1] *
                        (self.transition_reward[s, a, s1] + self.gamma * V[s1])
                        for s1 in range(self.num_states)
                    )
                policy[s] = np.argmax(q_values)
                if old_action != policy[s]:
                    policy_stable = False
        return policy, V, total_iterations

    def value_itr(self, theta=1e-4, max_iter=1000):
        V = np.zeros(self.num_states)
        iterations = 0
        for _ in range(max_iter):
            iterations += 1
            delta = 0
            for s in range(self.num_states):
                if s == self.num_states - 1:  # Skip terminal state
                    continue
                v_old = V[s]
                max_q = -np.inf
                for a in range(self.num_actions):
                    q = sum(
                        self.transition_prob[s, a, s1] *
                        (self.transition_reward[s, a, s1] + self.gamma * V[s1])
                        for s1 in range(self.num_states)
                    )
                    max_q = max(max_q, q)
                V[s] = max_q
                delta = max(delta, abs(v_old - V[s]))
            if delta < theta:
                break

        # Derive policy
        policy = np.zeros(self.num_states, dtype=int)
        for s in range(self.num_states):
            q_values = np.zeros(self.num_actions)
            for a in range(self.num_actions):
                q_values[a] = sum(
                    self.transition_prob[s, a, s1] *
                    (self.transition_reward[s, a, s1] + self.gamma * V[s1])
                    for s1 in range(self.num_states)
                )
            policy[s] = np.argmax(q_values)
        return policy, V, iterations

    def visualize_grid(self):
        """Create a visual representation of the grid with obstacles and goal"""
        grid = np.full((self.grid_size, self.grid_size), '.', dtype='U4')
        grid[self.grid_obstacles] = 'X'
        grid[self.goal] = 'G'
        grid[0, 0] = 'S'  # Start state
        return grid

if __name__ == "__main__":
    np.random.seed(42)  # For reproducibility
    grid_size = 16
    env = GridWorld(grid_size)

    # Show grid layout
    print("Grid Layout:")
    visual_grid = env.visualize_grid()
    for row in visual_grid:
        print(' '.join(row))
    print("\n")

    # Run Policy Iteration
    pi_policy, pi_values, pi_iterations = env.policy_itr()
    print("Policy Iteration Results:")
    print(f"Total iterations: {pi_iterations}")
    print("Optimal Policy (0=UP, 1=DOWN, 2=LEFT, 3=RIGHT):")
    print(pi_policy.reshape(grid_size, grid_size))
    print("Value Function:")
    print(pi_values.reshape(grid_size, grid_size).round(2))
    print("\n")

    # Run Value Iteration
    vi_policy, vi_values, vi_iterations = env.value_itr()
    print("Value Iteration Results:")
    print(f"Total iterations: {vi_iterations}")
    print("Optimal Policy (0=UP, 1=DOWN, 2=LEFT, 3=RIGHT):")
    print(vi_policy.reshape(grid_size, grid_size))
    print("Value Function:")
    print(vi_values.reshape(grid_size, grid_size).round(2))
    print("\n")

    # Compare methods
    if pi_iterations < vi_iterations:
        print(f"Policy Iteration was faster ({pi_iterations} iterations vs {vi_iterations})")
    elif pi_iterations > vi_iterations:
        print(f"Value Iteration was faster ({vi_iterations} iterations vs {pi_iterations})")
    else:
        print(f"Both methods took the same number of iterations ({pi_iterations})")

Grid Layout:
S . . . . . . X . . X X . . . .
X . . X X . . . . X X . . . . X
. . X . . . . . . . . . . . . .
. . . . . . . . . . . . . . . .
. . . . . X . . . . . . . . . .
X . . . . . . X . . . . . . . .
. X X . . . . . . . . . . . . .
. . . . . . X . . X . . . X . .
. . . . . . . . . X . . . . . .
. . . . . X . . . X . . . . . .
. . . . . X . . X . . . . . . .
. . . . . X . . . X X . . . . X
. . . . . . . X . . X . . . X .
. . X . . . . . . X . . . X X .
. . . . . X . . . . X . . . . .
. . . X . . . . . . . . X . . G


Policy Iteration Results:
Total iterations: 543
Optimal Policy (0=UP, 1=DOWN, 2=LEFT, 3=RIGHT):
[[3 1 3 3 3 1 1 1 1 2 2 1 1 1 1 2]
 [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1]
 [3 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1]
 [3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 3 3 3 3 3 3 3 3 1 1 1 2 1 1]
 [1 1 1 0 0 0 0 0 0 3 1 1 1 1 1 1]
 [1 1 1 1 1 3 1 1 0 3 1 1 1 1 1 1]
 [1 1 1 1 1 3 1 1 2 1 1 1 1 1 1 1]
 [1 1 1 1 1 3 1 1 3 3 3 1 1

- **Setup**: A 16x16 grid with 15% random obstacles (marked 'X'), a start at (0,0) ('S'), goal at (15,15) ('G'), and a -0.1 penalty for hitting obstacles.
- **Algorithms**: Enhanced to handle obstacles via transition and reward matrices.
- **Results**:
  - **Policy Iteration**: Takes 543 iterations, optimal policy navigates around obstacles toward the goal, value function reflects penalties (e.g., dips near obstacles) and peaks at 1.0.
  - **Value Iteration**: Converges in 31 iterations, yielding an almost identical policy and value function.
  - Grid visualization shows obstacle placement, with policies adapting to avoid 'X' cells.
- **Comparison**: Value Iteration remains faster (31 vs. 543 iterations), despite the added complexity of obstacles.


### Key Insights
- **Environment Complexity**: The experiment progresses from a simple 3x3 grid to a 16x16 grid with obstacles, testing algorithm robustness.
- **Algorithm Performance**:
  - Iterative Policy Evaluation is limited to fixed policies and serves as a baseline.
  - Policy Iteration, while effective, requires more iterations due to repeated policy evaluation.
  - Value Iteration consistently converges faster by directly optimizing the value function.
- **Convergence**: Both Policy and Value Iteration find optimal policies, but Value Iteration’s single-pass approach outperforms in larger, complex grids.
- **Practicality**: Obstacles introduce realistic navigation challenges, and the penalty (-0.1) shapes policies to avoid them, validated by value function dips.

###*The experiment demonstrates that Value Iteration is generally more efficient than Policy Iteration in terms of iterations, especially in larger grids or with obstacles, making it preferable for computational efficiency. All algorithms successfully solve the GridWorld problem, with results aligning with reinforcement learning theory—higher values and directed policies toward the goal.*